##Connection to Storage

In [0]:
spark.conf.set("fs.azure.account.auth.type.'storage-account'.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.'storage-account'.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.'storage-account'.dfs.core.windows.net", 'application-id')
spark.conf.set("fs.azure.account.oauth2.client.secret.'storage-account'.dfs.core.windows.net", 'service_credential')
spark.conf.set("fs.azure.account.oauth2.client.endpoint.'storage-account'.dfs.core.windows.net", "https://login.microsoftonline.com/'directory-id'/oauth2/token")

In [0]:
%sql
USE CATALOG hive_metastore;


In [0]:
%sql
SELECT current_catalog();


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS gold;


##Importing Required Functions

In [0]:
from pyspark.sql.functions import *

# Data Reading & Writing & CREATING Delta Tables

In [0]:
silver = 'abfss://silver@nyctaxivamsi.dfs.core.windows.net'
gold = 'abfss://gold@nyctaxivamsi.dfs.core.windows.net'

####trip_zone

In [0]:
trip_zone_df = spark.read.format("parquet")\
                        .option("header", "true")\
                        .option("inferschema", True)\
                        .load(f"{silver}/trip_zone")

In [0]:
display(trip_zone_df.limit(50))

In [0]:
trip_zone_df.write.format('delta')\
                    .mode('overwrite')\
                    .option('path',f'{gold}/trip_zone')\
                    .save()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.trip_zone
USING DELTA
LOCATION 'abfss://gold@nyctaxivamsi.dfs.core.windows.net/trip_zone';


In [0]:
%sql
select *
from gold.trip_zone
limit 50;

####trip_type

In [0]:
trip_type_df = spark.read.format("parquet")\
                        .option("header", "true")\
                        .option("inferschema", True)\
                        .load(f"{silver}/trip_type")

In [0]:
display(trip_type_df.limit(50))

In [0]:
trip_type_df.write.format('delta')\
                    .mode('overwrite')\
                    .option('path',f'{gold}/trip_type')\
                    .save()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.trip_type
USING DELTA
LOCATION 'abfss://gold@nyctaxivamsi.dfs.core.windows.net/trip_type';

In [0]:
%sql
select *
from gold.trip_type

####trip_data

In [0]:
trip_data_df = spark.read.format("parquet")\
                        .option("header", "true")\
                        .option("inferschema", True)\
                        .load(f"{silver}/trip_data")

In [0]:
display(trip_data_df.limit(50))

In [0]:
trip_data_df.write.format('delta')\
                    .mode('overwrite')\
                    .option('path',f'{gold}/trip_data')\
                    .save()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.trip_data
USING DELTA
LOCATION 'abfss://gold@nyctaxivamsi.dfs.core.windows.net/trip_data';

In [0]:
%sql
select * 
from gold.trip_data
limit 50;

# Learning Delta Lake

**Versioning**

In [0]:
%sql 
select * from gold.trip_zone WHERE LocationID=1

In [0]:
%sql
UPDATE gold.trip_zone
SET Borough='EMR'
WHERE LocationID=1

In [0]:
%sql
DELETE FROM gold.trip_zone WHERE locationID=1

In [0]:
%sql 
DESCRIBE HISTORY gold.trip_zone

**Time Travel**

In [0]:
%sql
RESTORE gold.trip_zone TO VERSION AS OF 0;

# Final data viewing 
##project completion

####trip_zone

In [0]:
%sql
select * from gold.trip_zone;

####trip_type

In [0]:
%sql
select * from gold.trip_type;

####trip_data

In [0]:
%sql
select * from gold.trip_data;